In [71]:
import pandas as pd
import numpy as np

from catboost import CatBoostRegressor
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from datetime import datetime

pd.set_option('display.max_rows', 5000)

zhi_df2=pd.read_csv('../data/Zip_Zri_MultiFamilyResidenceRental.csv')


zri = pd.melt(zhi_df2, id_vars = zhi_df2.columns[0:6], value_vars = zhi_df2.columns[7:], var_name = 'year-month', value_name = 'ZRI').sort_values(['RegionName', 'year-month'])

#zri=zri[zri['year-month']>'2014-02']

zri=zri.fillna(0)
zri['gapsize']=zri['ZRI'].rolling(6).sum()
droplist=list(zri[(zri['gapsize']==0) & (zri['year-month']>'2014-06')]['RegionName'].drop_duplicates())
zri[(zri['gapsize']==0)].shape[0]/zri.shape[0]
len(droplist)/len(zri['RegionName'].drop_duplicates())

0.30037614185921546

In [72]:
zhi_df2.shape

(1861, 120)

In [73]:
date_mapping={i:j for i,j in enumerate(zhi_df.columns[6:])}

In [84]:
zri2 = pd.melt(zhi_df2, id_vars = zhi_df2.columns[0:6], value_vars = zhi_df2.columns[7:], var_name = 'year-month', value_name = 'ZRI').sort_values(['RegionName', 'year-month'])

zri2[(zri2['year-month']=='2019-12') & (zri2['ZRI'].isna())]=zri2[(zri2['year-month']=='2019-12') & (zri2['ZRI'].isna())].fillna(method='ffill')

zri2=zri2.groupby('RegionName').apply(lambda group: group.interpolate())
#zri2[zri2['ZRI'].isna()]

#zri2['ZRI']=(zri2['ZRI']/zri2['ZRI'].shift(1))-1

zri2=zri2[(zri2['year-month']>'2014-01') & (zri2['year-month']<'2020-01')]

zri3=zri2[~(zri2['RegionName'].isin(droplist))]

zri3['ZRI']=zri3['ZRI'].fillna(method='bfill')



<ipython-input-84-585f0312ac37>:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [85]:
#zip codes to drop
len(droplist)

559

In [86]:
#remaining filterd ZRI data:
len(zri3['RegionName'].drop_duplicates())

1302

In [87]:
zri3.isna().sum()

RegionID       0
RegionName     0
City           0
State          0
Metro         70
CountyName     0
year-month     0
ZRI            0
dtype: int64

<H1>Lag development

In [89]:
zri3=zri3.drop('RegionID', axis=1)

In [90]:
def laggenerator(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i]
    df.loc[:,f'{colname}_lag{i}']=df.loc[:,f'{colname}'].shift(i)
    df.loc[df['year-month'].isin(timelist),f'{colname}_lag{i}']=0
    return df

In [99]:
def laggenerator_diff(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i+1]
    df.loc[:,f'{colname}_diff_lag{i}_lag{i-1}']=df.loc[:,f'{colname}'].shift(i)-df.loc[:,f'{colname}'].shift(i+1)
    df.loc[df['year-month'].isin(timelist),f'{colname}_lag{i}']=0
    return df

In [102]:
zri3=laggenerator(1, 'ZRI',zri3)
zri3=laggenerator(2, 'ZRI',zri3)
zri3=laggenerator(3, 'ZRI',zri3)
zri3=laggenerator(4, 'ZRI',zri3)
zri3=laggenerator(5, 'ZRI',zri3)
zri3=laggenerator(6, 'ZRI',zri3)

zri3=laggenerator_diff(0, 'ZRI',zri3)

In [103]:
zri3

,RegionName,City,State,Metro,CountyName,year-month,ZRI,ZRI_lag1,ZRI_lag2,ZRI_lag3,ZRI_lag4,ZRI_lag5,ZRI_lag6,ZRI_diff_lag6_lag5,ZRI_diff_lag0_lag-1,ZRI_lag0
77798,1013,Chicopee,MA,Springfield,Hampden County,2014-02,931.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0
79659,1013,Chicopee,MA,Springfield,Hampden County,2014-03,934.0,931.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,NaN
81520,1013,Chicopee,MA,Springfield,Hampden County,2014-04,929.0,934.0,931.0,0.0,0.0,0.0,0.0,NaN,-5.0,NaN
83381,1013,Chicopee,MA,Springfield,Hampden County,2014-05,929.0,929.0,934.0,931.0,0.0,0.0,0.0,NaN,0.0,NaN
85242,1013,Chicopee,MA,Springfield,Hampden County,2014-06,929.0,929.0,929.0,934.0,931.0,0.0,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200463,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-08,1277.0,1291.0,1294.0,1296.0,1307.0,1318.0,1321.0,-8.0,-14.0,NaN
202324,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-09,1271.0,1277.0,1291.0,1294.0,1296.0,1307.0,1318.0,-3.0,-6.0,NaN
204185,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-10,1299.0,1271.0,1277.0,1291.0,1294.0,1296.0,1307.0,-11.0,28.0,NaN
206046,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-11,1261.5,1299.0,1271.0,1277.0,1291.0,1294.0,1296.0,-11.0,-37.5,NaN


In [92]:
train, test=zri3[(zri3['year-month']<='2018-12') & (zri3['year-month']>'2014-06')], zri3[(zri3['year-month']>'2018-12') & (zri3['year-month']<='2019-12')]

train=train.drop('year-month',axis=1)

Y_train, Y_test=train['ZRI'], test[['ZRI']]
X_train, X_test=train.drop('ZRI',axis=1), test.drop('ZRI',axis=1)



In [93]:
labelencoder = LabelEncoder()

X_train['CountyNameEnc'] = labelencoder.fit_transform(X_train['CountyName'])

X_train=X_train.drop(['CountyName','City','State','Metro'],axis=1)


In [94]:
cat=CatBoostRegressor()

In [95]:
cat.fit(X_train, Y_train)

Learning rate set to 0.083208
0:	learn: 611.4762699	total: 13.3ms	remaining: 13.3s
1:	learn: 562.9199603	total: 17.8ms	remaining: 8.87s
2:	learn: 518.7212919	total: 22.7ms	remaining: 7.56s
3:	learn: 477.9461357	total: 27.7ms	remaining: 6.9s
4:	learn: 440.2126203	total: 33ms	remaining: 6.57s
5:	learn: 405.8628481	total: 37.8ms	remaining: 6.26s
6:	learn: 374.4239104	total: 42.7ms	remaining: 6.06s
7:	learn: 345.2496098	total: 48.2ms	remaining: 5.98s
8:	learn: 318.4257615	total: 54.1ms	remaining: 5.96s
9:	learn: 293.8815206	total: 59.7ms	remaining: 5.91s
10:	learn: 271.1571943	total: 64.2ms	remaining: 5.77s
11:	learn: 250.3639211	total: 69ms	remaining: 5.68s
12:	learn: 231.5016313	total: 73.7ms	remaining: 5.59s
13:	learn: 213.8148008	total: 78.2ms	remaining: 5.51s
14:	learn: 197.6730241	total: 82.3ms	remaining: 5.41s
15:	learn: 183.2398913	total: 88.7ms	remaining: 5.46s
16:	learn: 169.7091416	total: 93.3ms	remaining: 5.39s
17:	learn: 157.4358135	total: 97.8ms	remaining: 5.34s
18:	learn: 14

194:	learn: 26.7971960	total: 961ms	remaining: 3.97s
195:	learn: 26.7715091	total: 965ms	remaining: 3.96s
196:	learn: 26.7307067	total: 970ms	remaining: 3.95s
197:	learn: 26.6902694	total: 975ms	remaining: 3.95s
198:	learn: 26.6649415	total: 979ms	remaining: 3.94s
199:	learn: 26.6152882	total: 984ms	remaining: 3.94s
200:	learn: 26.5744344	total: 989ms	remaining: 3.93s
201:	learn: 26.5384255	total: 994ms	remaining: 3.92s
202:	learn: 26.4768114	total: 999ms	remaining: 3.92s
203:	learn: 26.4348086	total: 1s	remaining: 3.92s
204:	learn: 26.3868621	total: 1.01s	remaining: 3.92s
205:	learn: 26.3456475	total: 1.01s	remaining: 3.91s
206:	learn: 26.3245593	total: 1.02s	remaining: 3.9s
207:	learn: 26.2924217	total: 1.02s	remaining: 3.9s
208:	learn: 26.2185993	total: 1.03s	remaining: 3.9s
209:	learn: 26.1822583	total: 1.03s	remaining: 3.89s
210:	learn: 26.1256738	total: 1.04s	remaining: 3.89s
211:	learn: 26.1065901	total: 1.04s	remaining: 3.88s
212:	learn: 26.0542544	total: 1.05s	remaining: 3.88s

385:	learn: 21.6221718	total: 1.94s	remaining: 3.09s
386:	learn: 21.6032079	total: 1.95s	remaining: 3.09s
387:	learn: 21.5869651	total: 1.95s	remaining: 3.08s
388:	learn: 21.5774424	total: 1.96s	remaining: 3.08s
389:	learn: 21.5637190	total: 1.96s	remaining: 3.07s
390:	learn: 21.5456991	total: 1.97s	remaining: 3.07s
391:	learn: 21.5259160	total: 1.97s	remaining: 3.06s
392:	learn: 21.5212172	total: 1.98s	remaining: 3.06s
393:	learn: 21.5121034	total: 1.98s	remaining: 3.05s
394:	learn: 21.4993465	total: 1.99s	remaining: 3.04s
395:	learn: 21.4802794	total: 1.99s	remaining: 3.04s
396:	learn: 21.4703532	total: 2s	remaining: 3.03s
397:	learn: 21.4531326	total: 2s	remaining: 3.03s
398:	learn: 21.4464372	total: 2.01s	remaining: 3.02s
399:	learn: 21.4304077	total: 2.01s	remaining: 3.02s
400:	learn: 21.4176982	total: 2.02s	remaining: 3.01s
401:	learn: 21.4104206	total: 2.02s	remaining: 3s
402:	learn: 21.4049935	total: 2.02s	remaining: 3s
403:	learn: 21.3814792	total: 2.03s	remaining: 2.99s
404:	

548:	learn: 19.8381796	total: 2.73s	remaining: 2.24s
549:	learn: 19.8303914	total: 2.74s	remaining: 2.24s
550:	learn: 19.8193375	total: 2.74s	remaining: 2.23s
551:	learn: 19.8090588	total: 2.75s	remaining: 2.23s
552:	learn: 19.7981452	total: 2.75s	remaining: 2.23s
553:	learn: 19.7878114	total: 2.76s	remaining: 2.22s
554:	learn: 19.7821897	total: 2.76s	remaining: 2.22s
555:	learn: 19.7721265	total: 2.77s	remaining: 2.21s
556:	learn: 19.7588994	total: 2.77s	remaining: 2.21s
557:	learn: 19.7488729	total: 2.78s	remaining: 2.2s
558:	learn: 19.7454791	total: 2.78s	remaining: 2.19s
559:	learn: 19.7389130	total: 2.79s	remaining: 2.19s
560:	learn: 19.7272449	total: 2.79s	remaining: 2.18s
561:	learn: 19.7190019	total: 2.8s	remaining: 2.18s
562:	learn: 19.7103587	total: 2.8s	remaining: 2.17s
563:	learn: 19.7018454	total: 2.81s	remaining: 2.17s
564:	learn: 19.6915930	total: 2.81s	remaining: 2.17s
565:	learn: 19.6851388	total: 2.82s	remaining: 2.16s
566:	learn: 19.6772170	total: 2.82s	remaining: 2.

705:	learn: 18.9003090	total: 3.52s	remaining: 1.46s
706:	learn: 18.8940904	total: 3.52s	remaining: 1.46s
707:	learn: 18.8900188	total: 3.53s	remaining: 1.45s
708:	learn: 18.8846737	total: 3.53s	remaining: 1.45s
709:	learn: 18.8784974	total: 3.54s	remaining: 1.44s
710:	learn: 18.8744010	total: 3.54s	remaining: 1.44s
711:	learn: 18.8706345	total: 3.55s	remaining: 1.43s
712:	learn: 18.8673650	total: 3.55s	remaining: 1.43s
713:	learn: 18.8638448	total: 3.56s	remaining: 1.42s
714:	learn: 18.8606656	total: 3.56s	remaining: 1.42s
715:	learn: 18.8574727	total: 3.56s	remaining: 1.41s
716:	learn: 18.8513798	total: 3.57s	remaining: 1.41s
717:	learn: 18.8446894	total: 3.58s	remaining: 1.4s
718:	learn: 18.8404980	total: 3.58s	remaining: 1.4s
719:	learn: 18.8373808	total: 3.58s	remaining: 1.39s
720:	learn: 18.8307576	total: 3.59s	remaining: 1.39s
721:	learn: 18.8293660	total: 3.59s	remaining: 1.38s
722:	learn: 18.8236656	total: 3.6s	remaining: 1.38s
723:	learn: 18.8195275	total: 3.6s	remaining: 1.3

868:	learn: 18.3138417	total: 4.3s	remaining: 648ms
869:	learn: 18.3114928	total: 4.3s	remaining: 643ms
870:	learn: 18.3071482	total: 4.31s	remaining: 638ms
871:	learn: 18.3037845	total: 4.31s	remaining: 633ms
872:	learn: 18.3016627	total: 4.32s	remaining: 628ms
873:	learn: 18.2982782	total: 4.32s	remaining: 623ms
874:	learn: 18.2973457	total: 4.33s	remaining: 618ms
875:	learn: 18.2939816	total: 4.33s	remaining: 613ms
876:	learn: 18.2916033	total: 4.33s	remaining: 608ms
877:	learn: 18.2890398	total: 4.34s	remaining: 603ms
878:	learn: 18.2861975	total: 4.34s	remaining: 598ms
879:	learn: 18.2825894	total: 4.35s	remaining: 593ms
880:	learn: 18.2805107	total: 4.35s	remaining: 588ms
881:	learn: 18.2786659	total: 4.36s	remaining: 583ms
882:	learn: 18.2747635	total: 4.36s	remaining: 578ms
883:	learn: 18.2715653	total: 4.37s	remaining: 573ms
884:	learn: 18.2682832	total: 4.37s	remaining: 568ms
885:	learn: 18.2657771	total: 4.38s	remaining: 563ms
886:	learn: 18.2623089	total: 4.38s	remaining: 5

In [96]:
#loop months
#for January: filter Jan
#predict Jan
#update all lags
#loop

predictor_table=zri3.copy()

#loop through all month in 2019
for month in list(X_test['year-month'].drop_duplicates()):
    X_test2=X_test.copy()
    
    #run prediction for one month
    X_test2=X_test2[X_test2['year-month']==month]
    
    X_test2['CountyNameEnc'] = labelencoder.transform(X_test2['CountyName'])
    
    X_test2=X_test2.drop(['CountyName','City','State','Metro','year-month'],axis=1)
    val=cat.predict(X_test2)
    
    #define nextmonth
    nextmonth=date_mapping[[mon_ind for mon_ind, name in date_mapping.items() if name == month][0]+1]

    #write current month prediction into predictor_table
    if nextmonth<'2020-01':
        predictor_table.loc[predictor_table['year-month']==nextmonth,'ZRI']=val
        
        predictor_table=laggenerator(1, 'ZRI',predictor_table)
        predictor_table=laggenerator(2, 'ZRI',predictor_table)
        predictor_table=laggenerator(3, 'ZRI',predictor_table)
        predictor_table=laggenerator(4, 'ZRI',predictor_table)
        predictor_table=laggenerator(5, 'ZRI',predictor_table)
        predictor_table=laggenerator(6, 'ZRI',predictor_table)
        
        X_test=predictor_table[predictor_table['year-month']>'2018-12']



In [97]:
RSME=(sum((Y_test['ZRI']-X_test['ZRI'])**2)/len(Y_test))**0.5
RSME

60.59562040153776

In [98]:
R2=1-sum((Y_test['ZRI']-X_test['ZRI'])**2)/sum((Y_test['ZRI']-Y_test.mean()[0])**2)
R2

0.9917269277727773

In [63]:
Y_test.mean()[0]

0.0015732354761700603

In [17]:
merged=X_test.merge(Y_test, left_index=True, right_index=True)

In [18]:
import plotly.express as px

In [36]:
px.line(zri2.iloc[0:700], x='year-month', y='ZRI')

In [45]:
len(merged['RegionID'].drop_duplicates())

1266

In [40]:
merged

,RegionID,RegionName,City,State,Metro,CountyName,year-month,ZRI_x,ZRI_lag1,ZRI_lag2,ZRI_lag3,ZRI_lag4,ZRI_lag5,ZRI_lag6,ZRI_y
187597,58207,1013,Chicopee,MA,Springfield,Hampden County,2019-01,-0.009017,-0.005381,-0.008007,-0.012302,-0.008711,-0.003472,-0.003460,-0.009017
189458,58207,1013,Chicopee,MA,Springfield,Hampden County,2019-02,0.000834,-0.009017,-0.005381,-0.008007,-0.012302,-0.008711,-0.003472,-0.000910
191319,58207,1013,Chicopee,MA,Springfield,Hampden County,2019-03,-0.002216,0.000834,-0.009017,-0.005381,-0.008007,-0.012302,-0.008711,0.007286
193180,58207,1013,Chicopee,MA,Springfield,Hampden County,2019-04,0.005303,-0.002216,0.000834,-0.009017,-0.005381,-0.008007,-0.012302,0.007233
195041,58207,1013,Chicopee,MA,Springfield,Hampden County,2019-05,0.003683,0.005303,-0.002216,0.000834,-0.009017,-0.005381,-0.008007,0.001795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200523,399713,60642,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2019-08,0.007367,-0.002414,0.008446,-0.004490,0.004763,-0.011502,0.004144,0.003511
202384,399713,60642,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2019-09,-0.001426,0.007367,-0.002414,0.008446,-0.004490,0.004763,-0.011502,0.007996
204245,399713,60642,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2019-10,0.005973,-0.001426,0.007367,-0.002414,0.008446,-0.004490,0.004763,-0.005454
206106,399713,60642,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2019-11,-0.000652,0.005973,-0.001426,0.007367,-0.002414,0.008446,-0.004490,0.004985


In [21]:
#enc = OneHotEncoder(handle_unknown='ignore')

#X_train_enc = pd.DataFrame(enc.fit_transform(X_train[['CountyName']]).toarray())

#X_train = X_train.join(X_train_enc)
#X_train

#,

In [22]:
testzip=zri3[zri3['RegionID']==753844]
testzip['year-month']=pd.to_datetime(testzip['year-month'])
testzip=testzip.set_index('year-month')
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.express as px

decomposed=seasonal_decompose(testzip['ZRI'],model='additive')

trend=decomposed.trend
seasonal=decomposed.seasonal
residual=decomposed.resid

#px.line(testzip['ZRI'])
#px.line(trend)
#px.line(seasonal)
#px.line(residual)

ValueError: You must specify a period or x must be a pandas object with a DatetimeIndex with a freq not set to None

In [ ]:
from pmdarima.arima import auto_arima

x=100
X_train, X_test=testzip[0:x], testzip[x:len(testzip)-1]

arima_model=auto_arima(X_train['ZRI'], start_p=1, start_q=1, max_p=5, max_q=5, m=12, start_P=0, start_D=0,
                       start_Q=0, max_P=5, max_D=5, max_Q=5, 
                       seasonal=True, trace=True, stepwise=True, n_fits=50)

In [ ]:
zhi_reshaped2.gapsize==0

In [ ]:
zhi_reshaped2[(zhi_reshaped2['gapsize']==0)]

In [ ]:
zhi_reshaped2.fillna(0)
zhi_reshaped2['gapsize']=zhi_reshaped2['Value'].rolling(1).sum()

In [ ]:
zhi_reshaped2['gapsize']

In [ ]:
zhi_df.isna().sum(axis=1).shape

In [ ]:
zhi_df.isna().sum(axis=1)<1

In [ ]:
zhi_df[(zhi_df.isna().sum(axis=1)>8)].shape

In [ ]:
zhi_reshaped.groupby('RegionID').agg('count')

In [ ]:
zhi_reshaped

In [ ]:
acs_df=pd.read_csv('../data/acs_5yr_merged.csv')

In [ ]:
acs_df.shape

In [ ]:
acs_df.isna().sum()